##### 问题:
我们想将“私有”数据封装到类的实例上，但是又需要考虑到Python缺乏对属性的访问控制问题。

##### 解决方案:
与其依赖语言特性来封装数据，Python程序员们更期望通过特定的命名规则来表达出对数据和方法的用途。第一个规则是任何以单下划线（_）开头的名字应该总是被认为只属于内部实现。比如：

In [3]:
class A:
    def __init__(self):
        self._internal = 0  # An internal attribute
        self.public = 1     # A public attribute
    def public_method(self): 
        '''A public method'''
        pass
    def _internal_method(self):
        pass


Python本身并不会阻止其他人访问内部名称。但是如果有人这么做了，则被认为是粗鲁的，而且可能导致产生出脆弱不堪的代码。应该要提到的是，以下划线打头的标识也可用于模块名称和模块级的函数中。比如，如果见到有模块名以下划线打头（例如，_socket），那么它就属于内部实现。同样地，模块级的函数比如sys._getframe()使用起来也要格外小心。

我们应该在类定义中也见到过以双下划线（__）打头的名称。例如：

In [4]:
class B: 
    def __init__(self):
        self.__private = 0 
    def __private_method(self):
        pass
    def public_method(self):
        self.__private_method()



以双下划线打头的名称会导致出现名称重整（name  mangling）的行为。具体来说就是上面这个类中的私有属性会被分别重命名为_B__private和_B__private_method。此时你可能会问，类似这样的名称重整其目的何在？答案就是为了继承——这样的属性不能通过继承而覆盖。示例如下：

In [5]:
class C(B): 
    def __init__(self):
        super().__init__()
        self.__private = 1  # Does not override B.__private
    # Does not override B.__private_method()
    def __private_method(self):
        pass

这里，私有名称__private和__private_method会被重命名为_C__private和_C__ private_method，这和基类B中的重整名称不同。

“私有”属性存在两种不同的命名规则（单下划线和双下划线），这一事实引出了一个显而易见的问题：应该使用哪种风格？对于大部分代码而言，我们应该让非公有名称以单下划线开头。但是，如果我们知道代码中会涉及子类化处理，而且有些内部属性应该对子类进行隐藏，那么此时就应该使用双下划线开头。

此外还应该指出的是，有时候可能想定义一个变量，但是名称可能会和保留字产生冲突。基于此，应该在名称最后加上一个单下划线以示区别。比如：

lambda_ = 2.0  # Trailing _ to avoid clash with lambda keyword 

这里不采用以下划线开头的原因是避免在使用意图上发生混淆（例如，如果采用下划线开头的形式，那么可能会被解释为这么做是为了避免名称冲突，而不是作为私有数据的标志）。在名称尾部加一个单下划线就能解决这个问题。